### Google Collab Setup

In [ ]:
# TODO: Fill in as needed if you are using Google Collab or Locally

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import sys

# TODO: Fill in the Google Drive path where you uploaded the assignment
# Example: If you create a 2023WN folder and put all the files under P1 folder, then '2023WN/P1'
# GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = '2023WN/P1'
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'ROB599/Discussions/Demo/dis3_demo'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)

print(os.listdir(GOOGLE_DRIVE_PATH))

sys.path.append(GOOGLE_DRIVE_PATH)


### Import Packages

In [ ]:
import rob599
import torch
import torchvision
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['font.size'] = 16

### Visualize Data

In [ ]:
import random
from torchvision.utils import make_grid

num_train = 500
num_test = 250

x_train, y_train, x_test, y_test = rob599.data.progress_objects(num_train, num_test)

classes = [
    "master_chef_can",
    "cracker_box",
    "sugar_box",
    "tomato_soup_can",
    "mustard_bottle",
    "tuna_fish_can",
    "gelatin_box",
    "potted_meat_can",
    "mug",
    "large_marker"
]
samples_per_class = 12
samples = []
for y, cls in enumerate(classes):
    plt.text(-4, 34 * y + 18, cls, ha='right')
    idxs, = (y_train == y).nonzero(as_tuple=True)
    for i in range(samples_per_class):
        idx = idxs[random.randrange(idxs.shape[0])].item()
        samples.append(x_train[idx])
img = torchvision.utils.make_grid(samples, nrow=samples_per_class)
plt.imshow(rob599.tensor_to_image(img))
plt.axis('off')
plt.show()

### Setup Parameters

In [ ]:
BATCH_SIZE = 256
NUM_CLASSES = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
classes = [
    "master_chef_can",
    "cracker_box",
    "sugar_box",
    "tomato_soup_can",
    "mustard_bottle",
    "tuna_fish_can",
    "gelatin_box",
    "potted_meat_can",
    "mug",
    "large_marker"
]


### Load Data into Dataloader

In [ ]:
from torchvision.transforms import transforms

# Define the transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the train and test datasets
dset_train = rob599.ProgressObjectsDataset(root=".", train=True, transform=transform)
dset_test = rob599.ProgressObjectsDataset(root=".", train=False,transform=transform)

train_loader = torch.utils.data.DataLoader(dset_train, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dset_test, batch_size=BATCH_SIZE, shuffle=False)



### Define Training

In [ ]:
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    # Train the model for the specified number of epochs
    for epoch in range(num_epochs):
        # Set the model to train mode
        model.train()

        # Initialize the running loss and accuracy
        running_loss = 0.0
        running_corrects = 0

        # Iterate over the batches of the train loader
        for inputs, labels in train_loader:
            # Move the inputs and labels to the device
            labels = labels.type(torch.LongTensor)
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the optimizer gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # Backward pass and optimizer step
            loss.backward()
            optimizer.step()

            # Update the running loss and accuracy
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        # Calculate the train loss and accuracy
        train_loss = running_loss / len(train_loader.dataset)
        train_acc = running_corrects.double() / len(train_loader.dataset)

        # Set the model to evaluation mode
        model.eval()

        # Initialize the running loss and accuracy
        running_loss = 0.0
        running_corrects = 0

        # Iterate over the batches of the validation loader
        with torch.no_grad():
            for inputs, labels in val_loader:
                # Move the inputs and labels to the device
                labels = labels.type(torch.LongTensor)
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # Update the running loss and accuracy
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

        # Calculate the validation loss and accuracy
        val_loss = running_loss / len(val_loader.dataset)
        val_acc = running_corrects.double() / len(val_loader.dataset)

        # Print the epoch results
        print('Epoch [{}/{}], train loss: {:.4f}, train acc: {:.4f}, val loss: {:.4f}, val acc: {:.4f}'
              .format(epoch+1, num_epochs, train_loss, train_acc, val_loss, val_acc))


### Load Resnet

In [ ]:
import torch
import torchvision.models as models

# Load the pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)

# Freeze all the pre-trained layers
for param in model.parameters():
    param.requires_grad = False

# Modify the last layer of the model
model.fc = torch.nn.Linear(model.fc.in_features, NUM_CLASSES)

model.to(device)

### Stats without Training

In [ ]:
running_corrects = 0
train_acc = 0.0

# Set the model to evaluation mode
model.eval()

for inputs, labels in train_loader:
        # Move the inputs and labels to the device
        labels = labels.type(torch.LongTensor)
        inputs = inputs.to(device)
        labels = labels.to(device)
        # Forward pass
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        # Update the running loss and accuracy
        running_corrects += torch.sum(preds == labels.data)

    # Calculate the train loss and accuracy
train_acc = running_corrects.double() / len(train_loader.dataset)

running_corrects = 0

# Iterate over the batches of the validation loader

for inputs, labels in test_loader:
    # Move the inputs and labels to the device
    labels = labels.type(torch.LongTensor)
    inputs = inputs.to(device)
    labels = labels.to(device)

    # Forward pass
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)

    # Update the running loss and accuracy
    running_corrects += torch.sum(preds == labels.data)

# Calculate the validation loss and accuracy
val_acc = running_corrects.double() / len(test_loader.dataset)

# Print the epoch results
print('Without Training: train acc: {:.4f}, val acc: {:.4f}'.format( train_acc, val_acc))


### Define Loss and Optimizer

In [ ]:
# Define the loss function and optimizer
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


### Train the model

In [ ]:
# With all but the last layer frozen

model.to(device)
num_epochs = 1
optimizer = torch.optim.Adam(model.fc.parameters())
train(model, train_loader, test_loader, loss, optimizer, num_epochs=num_epochs, device = device)


### Show Results

In [ ]:
show_loader = torch.utils.data.DataLoader(dset_train, batch_size=1, shuffle=True)
dset_show = rob599.ProgressObjectsDataset(root=".", train=True, transform=transform)

model.eval()

for img, labels in show_loader:
    labels = labels.type(torch.LongTensor)
    img = img.to(device)
    labels = labels.to(device)
    outputs = model(img)
    _, preds = torch.max(outputs, 1)

    break
print(f"Predicted: {classes[preds.item()]}")
print(f"Actual: {classes[labels.item()]}")
plt.imshow(rob599.tensor_to_image(img.squeeze()))

### Optional Training

In [ ]:
# (optional) Retrain the whole network

for param in model.parameters():
    param.requires_grad = True

optimizer = torch.optim.Adam(model.fc.parameters())
train(model, train_loader, test_loader, loss, optimizer, num_epochs=num_epochs, device = device)


### With Resnet34

In [ ]:
import torch
import torchvision.models as models

# Load the pre-trained ResNet-34 model
model = models.resnet34(pretrained=True)

# Freeze all the pre-trained layers
for param in model.parameters():
    param.requires_grad = False

# Modify the last layer of the model
model.fc = torch.nn.Linear(model.fc.in_features, NUM_CLASSES)


In [ ]:
# With all but the last layer frozen

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 1
optimizer = torch.optim.Adam(model.fc.parameters())
train(model, train_loader, test_loader, loss, optimizer, num_epochs=num_epochs, device = device)
